In [1]:
from telethon.sync import TelegramClient
from telethon.tl.functions.contacts import ResolveUsernameRequest
from telethon.tl.functions.channels import GetMessagesRequest
from telethon.tl.functions.messages import GetHistoryRequest, ReadHistoryRequest
from telethon import functions, types
from datetime import date, datetime
from time import sleep
import os
import glob
from random import random
import re
import asyncio
from tqdm.notebook import tqdm
import pandas as pd
import json

### Этот блок нужно заполнить вашими значениями

In [2]:
# Замените на ваши данные, брать на my.telegram.org
api_id, api_hash = open('../../tg_api_id.txt', 'r').read().split()

my_channel = 'etonikmedoed'

In [3]:
client = TelegramClient('anon', api_id, api_hash)
await client.start()


def log_error(text):
    print(text)
    with open("log_err.txt", "a+", encoding="utf8") as f:
        f.write(f"{text}\n")

In [4]:
channel = await client.get_entity(my_channel)

In [5]:
class DateTimeEncoder(json.JSONEncoder):
    '''Класс для сериализации записи дат в JSON'''

    def default(self, o):
        if isinstance(o, datetime):
            return o.isoformat()
        if isinstance(o, bytes):
            return list(o)
        return json.JSONEncoder.default(self, o)


async def dump_all_messages(channel):
    """Записывает json-файл с информацией о всех сообщениях канала/чата"""
    offset_msg = 0    # номер записи, с которой начинается считывание
    limit_msg = 100   # максимальное число записей, передаваемых за один раз

    all_messages = []   # список всех сообщений
    total_messages = 0
    total_count_limit = 0  # поменяйте это значение, если вам нужны не все сообщения

    while True:
        history = await client(GetHistoryRequest(
            peer=channel,
            offset_id=offset_msg,
            offset_date=None, add_offset=0,
            limit=limit_msg, max_id=0, min_id=0,
            hash=0))
        if not history.messages:
            break
        messages = history.messages
        all_messages.extend(messages)
        offset_msg = messages[len(messages) - 1].id
        total_messages = len(all_messages)
        if total_count_limit != 0 and total_messages >= total_count_limit:
            break
    return all_messages

In [6]:
all_messages = await dump_all_messages(channel)

# with open('channel_messages.json', 'w', encoding='utf8') as outfile:
#      json.dump([message.to_dict() for message in all_messages:], outfile, ensure_ascii=False, cls=DateTimeEncoder)

In [7]:
entity_types = set()

for m in reversed(all_messages):
    for e in m.entities or []:
        entity_types.add(e.to_dict().get('_'))

entity_types

{'MessageEntityBold',
 'MessageEntityHashtag',
 'MessageEntityItalic',
 'MessageEntityStrike',
 'MessageEntityTextUrl',
 'MessageEntityUnderline'}

Если для типа не будет ниже функции, то будет использоваться дефолтная. Допишите необходмые функции.

In [8]:
MDformats = {
    'MessageEntity': "{text}",
    'MessageEntityBold': "<strong>{text}</strong>",
    'MessageEntityItalic': "<em>{text}</em>",
    'MessageEntityStrike': "<del>{text}</del>",
    'MessageEntityTextUrl': "[{text}]({entity.url})",
    'MessageEntityUnderline': "<ins>{text}</ins>"
}

In [12]:
csv_header = "Имя,Статус,Раздел,Теги,Пост в телеграм,Дата поста".split(
    ",")
sheet = pd.DataFrame(columns=csv_header)

for m in reversed(all_messages):
    if m.message:
        name = m.message.split('\n')[0].replace('&#8203;', '')
        tags = []

        text = []
        start = 0
        for e in m.entities or []:

            offset = e.offset
            while 0 < offset < len(m.message) and re.search("\s", m.message[offset]):
                offset += 1

            text.append(m.message[start:offset])

            start = e.offset+e.length
            while 0 < start < len(m.message) and re.search("\s", m.message[start-1]):
                start -= 1

            part = m.message[offset:start]

            etype = e.to_dict().get('_')
            if etype == 'MessageEntityHashtag':
                tags.append(part.replace('#', '').replace(
                    '_', ' ').capitalize())
                part = part.replace('#', '\#')
            text.append(MDformats.get(
                etype, MDformats['MessageEntity']).format(text=part, entity=e))
        text.append(m.message[start:])

        sheet.loc[len(sheet)] = [
            name,
            "Опубликовано",
            "Прочее",
            ", ".join(tags),
            f"https://t.me/{my_channel}/{m.id}",
            m.date.strftime("%B %d, %Y %H:%M")
        ]
        with open(f"postsMD/{m.id:04} - {re.sub('[:*?”<>|]','',name[:100])}.md", 'w', encoding='utf8') as f:
            text = "".join(text)
            text = re.sub('\n-', "\n\n-", text)
            text = re.sub('\n\*', "\n\n\*", text)
            f.write(text)

sheet.to_excel('posts.xlsx')